In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

gemini_api_key = settings.GEMINI_API_KEY

import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [40]:
import json
with open('data/002_output_15.json', 'r') as f:
    md15 = json.loads(f.read())
with open('data/002_output_36.json', 'r') as f:
    md41 = json.loads(f.read())

In [74]:
from sapl.materia.models import MateriaLegislativa
from cmj.utils import clean_text
reqs = [34, 36]
materias = MateriaLegislativa.objects.filter(numero__in=reqs, ano=2025, tipo_id=3)

texts = []
for materia in materias:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)
    texts.append(text)


In [75]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
#model = SentenceTransformer('ricardo-filho/bert-base-portuguese-cased-nli-assin-2')

model = SentenceTransformer('sentence-transformers/static-similarity-mrl-multilingual-v1')



#model = SentenceTransformer('neuralmind/bert-large-portuguese-cased')
A_embeddings = model.encode(texts[0])
B_embeddings = model.encode(texts[1])

In [43]:
import numpy as np
result = genai.embed_content(
    model='models/text-embedding-004',
    content=texts,
    task_type='semantic_similarity'
    )
"""A_embeddings = np.array(result['embedding'])

result = genai.embed_content(
    model='models/text-embedding-004',
    content=texts[1],
    task_type='similarity'
    )
B_embeddings = np.array(result['embedding'])"""
A_embeddings = result['embedding'][0]
B_embeddings = result['embedding'][1]

In [76]:
# Define the vectors
A = A_embeddings
B = B_embeddings

# Calculate cosine similarity
cos_similarity = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

# Calculate cosine distance
cos_distance = 1 - cos_similarity

print("Cosine Similarity: {:.2f}".format(cos_similarity))
print("Cosine Distance: {:.2f}".format(cos_distance))

Cosine Similarity: 0.62
Cosine Distance: 0.38


In [77]:
import numpy as np
from scipy.spatial.distance import cosine, euclidean

# Define the vectors
A = A_embeddings
B = B_embeddings

# Calculate cosine distance
cos_distance = cosine(A, B)  # Uses scipy's cosine for cosine distance

print("Cosine Distance: {:.2f}".format(cos_distance))

Cosine Distance: 0.38


In [56]:
print(texts[0])

Requerimento KC-04/2025 À MESA DIRETORA DA CÂMARA MUNICIPAL DE JATAÍ-GO
"Solicita à Secretaria de Obras e de Serviços Urbanos a revitalização da Praça Olho D’água."
A Vereadora Kátia Carvalho, que este subscreve, no uso de suas atribuições legais e regimentais, requer à Presidência desta Casa, após apreciação do plenário, o encaminhamento deste requerimento ao Ilmo. Sr. Cantimiro Melo Martins, Secretário de Obras, e ao Ilmo. Sr. Tiago Alves Rua, Secretário de Serviços Urbanos, solicitando a revitalização da Praça Olho D’água.
JUSTIFICATIVA
Trata-se de demanda recebida em meu gabinete, na qual moradores do município e dos arredores da referida praça se queixam, estando o local em condições de abandono, contendo bancos quebrados, falta de iluminação, parque infantil deteriorado, matagal alto e lixo nos lagos. Também, importante destacar que o lugar tem servido de ponto para uso de entorpecentes, tendo assim, vários usuários semanalmente no local, o que compromete a segurança dos moradore

In [57]:
print(texts[1])

EXCELENTÍSSIMO SENHOR PRESIDENTE DA CÂMARA MUNICIPAL DE JATAÍ, ESTADO DE GOIÁS.
Câmara Municipal de Jataí - GO, 03 de janeiro de 2025. Autor: Vereador Durval Gomes de Oliveira
"Solicita ao Poder Executivo que faça um estudo de viabilidade técnica para realizar a revitalização completa do Parque Ecológico Binômio da Costa Lima (Praça Olho d’água)."
O Parlamentar DURVAL GOMES DE OLIVEIRA, na forma regimental e no uso de suas atribuições legais, requer o encaminhamento de um exemplar deste ao Excelentíssimo Senhor GENEILTON FILHO DE ASSIS, DD. Prefeito Municipal de Jataí e ao Ilustríssimo Senhor CANTIMIRO MELO MARTINS, DD. Secretário de Obras, solicitando ao Poder Executivo que faça um estudo de viabilidade técnica para realizar a revitalização completa do Parque Ecológico Binômio da Costa Lima (Praça Olho d’água).
JUSTIFICATIVA
Ocorre, que o referido parque está completamente abandonado, carecendo de uma revitalização completa por toda sua extensão. As vegetações dentro dos lagos estão a